# 生成されたサンプルを辞書に基づいて日本語に直す

seqGANで生成されたサンプルを辞書に基づいて日本語の文字列に変換する

また、変換する際には、seqGANの識別機が算出したサンプルに対する本物かどうかの確率と紐付ける

これは、生成されたキャッチコピーに対する自然さの度合いを表していると考えられ、キャッチコピーに対するスコア付けの手法の一つとして用いる

今回は博多駅を対象にしており、他の観光地にて生成を行う場合は、適当に合わせること

In [45]:
# 生成されたサンプルを辞書に基づいて日本語に戻す
import pickle

# seqGANにて生成した結果を読み込む
with open('data/generated/dazaifu/output/output_text_200.txt') as f:
    s = f.readlines()

# seqGANの生成結果に対する識別器の確率を読み込む
with open('data/generated/dazaifu/output/prob_200.txt') as f:
    prob = f.readlines()

# 生成結果はid列なので日本語に戻すために作成した辞書を読み込む
with open('data/dict/all_dict.pickle', 'rb') as f:
    indices_char = pickle.load(f)

temp_list = []

for i, j in zip(s, prob):
    temp = [i,j]
    temp_list.append(temp)

print("生成数",len(temp_list))

num_prob_list = []

for k in temp_list:
    if k not in num_prob_list:
        num_prob_list.append(k)

print("重複をなくした生成数",len(num_prob_list))




# # 順番を保持したまま重複をなくす
# s = sorted(set(s), key=s.index)
# print(len(s))

haiku_prob_list = []
haiku_temp = []
result = ""
for j in range(len(num_prob_list)):
    line = num_prob_list[j][0].split()
    line = [int(i) for i in line]
    # print(line)
    # 単語に対応するidを並べて行く
    for i in line:
        if i > len(indices_char):
            continue
        # print(char_indices[i])
        result += str(indices_char[i])
        result += " "
    # result += "0" # 文末記号を追加
    result = result.replace('_', '')
    result = result.replace(' ', '')
    # result += "\n"
    haiku_temp.append(result)
    result = ""
# print(haiku_temp[:10])
for p in range(len(haiku_temp)):
    num_prob_list[p][1] = num_prob_list[p][1][:-1]
    temp = [haiku_temp[p],num_prob_list[p][1]]
    haiku_prob_list.append(temp)

print(len(haiku_prob_list))
print("\n")
print(haiku_prob_list[:10])


生成数 6400
重複をなくした生成数 604
604


[['有名なバリエーションの商店街もにぎわっました', '0.94820225'], ['お参りにたどり着くまで４時間', '0.6153461'], ['2月に途中一年半振りに', '0.53529686'], ['大好きな和菓子さんになる店が', '0.4433223'], ['参道も天満宮も参拝者', '0.43593347'], ['参道の梅が枝餅を購入し', '0.42193896'], ['いつ見ても９時前を', '0.3748845'], ['され失意のうちにました', '0.35776415'], ['境内の梅が枝餅を焼く自分', '0.31326932'], ['方々の観光客もにぎわっました', '0.29361945']]


日本語に変換したサンプルが575の音韻を満たしているかフィルタリングを行う

やってること自体は、レビューから575の文字列を抽出しているのと同じ

In [46]:
# with open('generated/sepa/hakataeki/content/save/output/output_2_haiku.txt') as f:
#     s = f.readlines()

import MeCab
tagger = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd") #mecab neologd


# text_list = []

# for line in s:
#     line = line.replace(" ", "")
#     text_list.append(line)

# print(text_list)




haiku_list = []
for x,y in haiku_prob_list:
    temp = x
    # print(temp)

    word_list = []

    # 形態素解析をして、単語毎にリストに格納
    for word in tagger.parse(temp).splitlines():
        word = word.replace('\t', ',')
        # print(word)
        temp_word_list = word.split(',')
        word_list.append(temp_word_list)

    # 文末の記号を削除
    word_list.pop(-1)

    # print(word_list)

    # 格納された単語を繋げて17音の文字列を作る
    for j in range(len(word_list)):

        # 最初に空の文字列を用意
        temp_haiku = ""
        part = ""
        # 次の単語を示すカウント
        count = 0
        # 音を数えるcount
        haku_count = 0

        # 17音を超えるまで単語を繋げる

        if not ('名詞' == word_list[j+count][1] or '動詞' == word_list[j+count][1] or '形容詞' == word_list[j+count][1]):
            continue
        if ('名詞' ==  word_list[j+count][1] and ('非自立' == word_list[j+count][2] or '接尾' == word_list[j+count][2])):
            continue
        if ('動詞' ==  word_list[j+count][1] and '非自立' == word_list[j+count][2]):
            continue

        # 上の句5
        while haku_count < 5:
            # 単語を繋げる
            part =part + word_list[j + count][0]
            # 音を記録

            if '記号' not in word_list[j+count][1]:
                temp = word_list[j+count][-1]

                # 小さい文字は音にならないので消去
                # temp = temp.replace('ッ', '')
                temp = temp.replace('ャ', '')
                temp = temp.replace('ュ', '')
                temp = temp.replace('ョ', '')
                temp = temp.replace('ァ', '')
                temp = temp.replace('ィ', '')
                temp = temp.replace('ゥ', '')
                temp = temp.replace('ェ', '')
                temp = temp.replace('ォ', '')

                haku_count += len(temp)
            # 元の文が終わっていたら終了
            if j + count >= len(word_list)-1:
                break
            count += 1
            # print(part, haku_count)
        
        if haku_count == 5:
            temp_haiku += part
            # temp_haiku += "\n"
            part = ""
            haku_count = 0
        else:
            continue

        if not ('名詞' == word_list[j+count][1] or '動詞' == word_list[j+count][1] or '形容詞' == word_list[j+count][1]):
            continue
        if ('名詞' ==  word_list[j+count][1] and ('非自立' == word_list[j+count][2] or '接尾' == word_list[j+count][2])):
            continue
        if ('動詞' ==  word_list[j+count][1] and '非自立' == word_list[j+count][2]):
            continue

        # 中の句7
        while haku_count < 7:
            # 単語を繋げる
            part =part + word_list[j + count][0]
            # 音を記録

            if '記号' not in word_list[j+count][1]:
                temp = word_list[j+count][-1]

                # 小さい文字は音にならないので消去
                # temp = temp.replace('ッ', '')
                temp = temp.replace('ャ', '')
                temp = temp.replace('ュ', '')
                temp = temp.replace('ョ', '')
                temp = temp.replace('ァ', '')
                temp = temp.replace('ィ', '')
                temp = temp.replace('ゥ', '')
                temp = temp.replace('ェ', '')
                temp = temp.replace('ォ', '')

                haku_count += len(temp)
            # 元の文が終わっていたら終了
            if j + count >= len(word_list)-1:
                break
            count += 1
            # print(temp_haiku, haku_count)

        
        
        if haku_count == 7:
            temp_haiku += part
            # temp_haiku += "\n"
            part = ""
            haku_count = 0
        else:
            continue

        if not ('名詞' == word_list[j+count][1] or '動詞' == word_list[j+count][1] or '形容詞' == word_list[j+count][1]):
            continue
        if ('名詞' ==  word_list[j+count][1] and ('非自立' == word_list[j+count][2] or '接尾' == word_list[j+count][2])):
            continue
        if ('動詞' ==  word_list[j+count][1] and '非自立' == word_list[j+count][2]):
            continue

        # 下の句5
        while haku_count < 5:
            # 単語を繋げる
            part =part + word_list[j + count][0]
            # 音を記録

            if '記号' not in word_list[j+count][1]:
                temp = word_list[j+count][-1]

                # 小さい文字は音にならないので消去
                # temp = temp.replace('ッ', '')
                temp = temp.replace('ャ', '')
                temp = temp.replace('ュ', '')
                temp = temp.replace('ョ', '')
                temp = temp.replace('ァ', '')
                temp = temp.replace('ィ', '')
                temp = temp.replace('ゥ', '')
                temp = temp.replace('ェ', '')
                temp = temp.replace('ォ', '')

                haku_count += len(temp)
            # 元の文が終わっていたら終了
            if j + count >= len(word_list)-1:
                break
            count += 1
            # print(temp_haiku, haku_count)

        
        if haku_count == 5:
            temp_haiku += part
            # temp_haiku += "\n"
            # temp_haiku += "\n"
            part = ""
            haku_count = 0
        else:
            continue
        

        if temp_haiku not in haiku_list:
            haiku_list.append([temp_haiku,y])

print(len(haiku_list))

print(haiku_list[:10])


156
[['参道も天満宮も参拝者', '0.43593347'], ['参道の梅が枝餅を購入し', '0.42193896'], ['境内の梅が枝餅を焼く自分', '0.31326932'], ['名物のクルーズ船が入ったと', '0.26894203'], ['参道を梅ヶ枝餅を食べながら', '0.23119384'], ['焼きもちを焼くので橋を渡るのは', '0.22360377'], ['参道の梅が枝餅を焼く香り', '0.19296506'], ['熱々の梅ヶ枝餅を食べるのが', '0.18865052'], ['お守りも買えます長い参道に', '0.18257838'], ['道中も気になる店が多かった', '0.17767288']]


# スコア付け

フィルタリングを行ったサンプルに対して、３つの手法でスコア付けを行う

- tf-idfを用いたスコア付け　⇨　観光地の特徴度を表す
- せqGANの識別器が算出したキャッチコピーに対する本物かどうかの確率を用いる　⇨　キャッチコピーの自然さを表す
- 先の２つのスコアを組み合わせたスコア付け　⇨　キャッチコピーの全体的な完成度を表す

# seqGANの識別器を用いたスコア付け

seqGANでは生成器が生成したキャッチコピーに対して、識別器がそれが本物かどうかの確率を算出する。

この時、識別器が本物だと判断すると、生成器は識別器を騙すことができており、自然なキャッチコピーが生成できていると考えられる。

したがって、識別器の確率は、キャッチコピーの自然さを表していると考えられる。

# tf-idfを用いたスコア付け

tf-idfを用いたスコア付けでは、サンプルに含まれる名詞に対するtf-idfの値の合計をキャッチコピーのスコアとする

これはキャッチコピーが持つ観光地の特徴度合いを表していると考えられる

In [47]:
#-------------------------------------------------データベース----------------------------------------

import sqlite3

db_word_list = []

conn = sqlite3.connect('data/tfidf_db.db')

c = conn.cursor()

c.execute('select word from dazaifu')
db_words = c.fetchall()

for word in db_words:
    db_word_list.append(word[0])

# print(db_word_list)

#-------------------------------------------------tfidfのスコア付け----------------------------------------
score_list = []

for i,_ in haiku_list:
    temp = i
    # print(temp)

    word_list = []
    score_sum = 0

    # 形態素解析をして、単語毎にリストに格納
    for word in tagger.parse(temp).splitlines():
        word = word.replace('\t', ',')
        temp_word_list = word.split(',')
        word_list.append(temp_word_list)

    # 文末の記号を削除
    word_list.pop(-1)

    

    for j in range(len(word_list)):
        # print(word_list[j][0])

        temp_score = 0

        if '名詞' not in word_list[j][1]:
            continue

        # 形態素解析した名詞がdbに含まれているならtfidfの値をスコアとして加算していく
        if word_list[j][0] in db_word_list:
            c.execute('select tfidf from dazaifu where word = ?', (word_list[j][0],))
            tfidf = c.fetchone()[0]
            temp_score = tfidf
        
        score_sum += temp_score

    score_list.append(score_sum)

        

# print(score_list)

#-------------------------------------------------データベース----------------------------------------

#閉じる
conn.close()





In [48]:
# 俳句とtfidfのスコアをまとめたリストを作成

for i in range(len(haiku_list)):
    haiku_list[i].append(score_list[i])

print("生成したキャッチコピー数：",len(haiku_list))
print("")
print("生成例")
for n in haiku_list[:10]:
    print("キャッチコピー:", n[0], " \t\t","識別器の確率:",n[1], "\t\t", "tf-idf値:",n[2])

# print(haiku_list)

# import pickle

# with open('data/generated/sepa/dazaifu/content/save/output/haiku_dis_tfidf.pickle', 'wb') as f:
#     pickle.dump(haiku_list, f)

生成したキャッチコピー数： 156

生成例
キャッチコピー: 参道も天満宮も参拝者  		 識別器の確率: 0.43593347 		 tf-idf値: 0.08266877790933559
キャッチコピー: 参道の梅が枝餅を購入し  		 識別器の確率: 0.42193896 		 tf-idf値: 0.07427657739892096
キャッチコピー: 境内の梅が枝餅を焼く自分  		 識別器の確率: 0.31326932 		 tf-idf値: 0.03896579286478265
キャッチコピー: 名物のクルーズ船が入ったと  		 識別器の確率: 0.26894203 		 tf-idf値: 0.0012811665595249965
キャッチコピー: 参道を梅ヶ枝餅を食べながら  		 識別器の確率: 0.23119384 		 tf-idf値: 0.08059833969157103
キャッチコピー: 焼きもちを焼くので橋を渡るのは  		 識別器の確率: 0.22360377 		 tf-idf値: 0.003547983760713978
キャッチコピー: 参道の梅が枝餅を焼く香り  		 識別器の確率: 0.19296506 		 tf-idf値: 0.07442992826495887
キャッチコピー: 熱々の梅ヶ枝餅を食べるのが  		 識別器の確率: 0.18865052 		 tf-idf値: 0.03006349179171362
キャッチコピー: お守りも買えます長い参道に  		 識別器の確率: 0.18257838 		 tf-idf値: 0.0631439911669297
キャッチコピー: 道中も気になる店が多かった  		 識別器の確率: 0.17767288 		 tf-idf値: 0.00022315096260099208


# 識別器の確率とtfidfを組み合わせた総合的なスコア付け

識別器の確率とtf-idfの値を組み合わせることで、キャッチコピーの総合的な完成度を表す。

組み合わせには、正規化した両方の値の調和平均を用いた。

In [49]:
import statistics

haiku_score_list = haiku_list

haiku_list_fin = []
tfidf_list = []
dis_list = []

for i in haiku_score_list:
    haiku_list_fin.append(i[0])
    dis_list.append(float(i[1]))
    tfidf_list.append(float(i[2]))

def min_max(l):
    l_min = min(l)
    l_max = max(l)
    return [(i - l_min) / (l_max - l_min) for i in l]

def standardization(l):
    l_mean = statistics.mean(l)
    l_stdev = statistics.stdev(l)
    return [(i - l_mean) / l_stdev for i in l]

# 標準化
# tfidf_list_std = standardization(tfidf_list)
# dis_list_std = standardization(dis_list)
# print(tfidf_list_std)

# 調和平均では値がマイナスだと困るため正規化を用いる
tfidf_list_std = min_max(tfidf_list)
dis_list_std = min_max(dis_list)
# print(tfidf_list_std)

調和平均を求める

In [50]:
sum_list = [0]*len(tfidf_list_std)

# 調和平均を計算
# 識別器の確率かtfidfのどちらかが0の場合は、調和平均を0にする
for i in range(len(tfidf_list_std)):
    if tfidf_list_std[i] == 0 or dis_list_std[i] == 0:
        sum_list[i] = 0
    else:
        sum_list[i] = 2/(1/tfidf_list_std[i] + 1/dis_list_std[i])

# print(sum_list)

生成例とそれに対するスコアを表示

識別器の確率とtfidfの値は、正規化されているため0~1で表示される

In [51]:
score_list = []
for i in range(len(sum_list)):
    temp = [haiku_list_fin[i],dis_list_std[i],tfidf_list_std[i],sum_list[i]]
    score_list.append(temp)

for n in score_list[:10]:
    print("生成例:", n[0], "\t", "識別器:", n[1], "\t", "tf-idf:", n[2], "\t", "調和平均:", n[3])

# スコアを保存
import pickle

with open('data/generated/dazaifu/score/score.pickle', 'wb') as f:
    pickle.dump(score_list, f)

生成例: 参道も天満宮も参拝者 	 識別器: 1.0 	 tf-idf: 1.0 	 調和平均: 1.0
生成例: 参道の梅が枝餅を購入し 	 識別器: 0.9675870764820672 	 tf-idf: 0.89848403807761 	 調和平均: 0.9317560696227312
生成例: 境内の梅が枝餅を焼く自分 	 識別器: 0.7158954681269851 	 tf-idf: 0.4713483596856502 	 調和平均: 0.5684361488401817
生成例: 名物のクルーズ船が入ったと 	 識別器: 0.6132282750257457 	 tf-idf: 0.015497586778530536 	 調和平均: 0.030231167459812757
生成例: 参道を梅ヶ枝餅を食べながら 	 識別器: 0.5257990477494502 	 tf-idf: 0.974955016027511 	 調和平均: 0.683163792654572
生成例: 焼きもちを焼くので橋を渡るのは 	 識別器: 0.5082195570487571 	 tf-idf: 0.04291806230164814 	 調和平均: 0.0791519135929894
生成例: 参道の梅が枝餅を焼く香り 	 識別器: 0.4372567177697741 	 tf-idf: 0.9003390415011042 	 調和平均: 0.588637174479906
生成例: 熱々の梅ヶ枝餅を食べるのが 	 識別器: 0.4272637380202108 	 tf-idf: 0.3636619840284178 	 調和平均: 0.39290561513000016
生成例: お守りも買えます長い参道に 	 識別器: 0.4131999509058224 	 tf-idf: 0.763819095477387 	 調和平均: 0.5362870103453379
生成例: 道中も気になる店が多かった 	 識別器: 0.4018382386117 	 tf-idf: 0.002699337866657784 	 調和平均: 0.00536265225692159


# 各スコアでの上位のキャッチコピー

各スコア付けでの上位１０件のキャッチコピーを表示

In [52]:
import pickle
with open('data/generated/dazaifu/score/score.pickle', 'rb') as f:
    score_list = pickle.load(f)

In [53]:
# 識別器の確率によるスコアでの上位のキャッチコピー

score_list_10 = sorted(score_list, key=lambda x: x[1])
score_list_10.reverse()
# score_list_10.pop(4)
score_list_10 = score_list_10[:10]

print("識別器の確率によるスコアでの上位のキャッチコピー")
print("")
for n in score_list_10:
    print("生成例:", n[0], "\t", "識別器:", n[1])

識別器の確率によるスコアでの上位のキャッチコピー

生成例: 参道も天満宮も参拝者 	 識別器: 1.0
生成例: 参道の梅が枝餅を購入し 	 識別器: 0.9675870764820672
生成例: 境内の梅が枝餅を焼く自分 	 識別器: 0.7158954681269851
生成例: 名物のクルーズ船が入ったと 	 識別器: 0.6132282750257457
生成例: 参道を梅ヶ枝餅を食べながら 	 識別器: 0.5257990477494502
生成例: 焼きもちを焼くので橋を渡るのは 	 識別器: 0.5082195570487571
生成例: 参道の梅が枝餅を焼く香り 	 識別器: 0.4372567177697741
生成例: 熱々の梅ヶ枝餅を食べるのが 	 識別器: 0.4272637380202108
生成例: お守りも買えます長い参道に 	 識別器: 0.4131999509058224
生成例: 道中も気になる店が多かった 	 識別器: 0.4018382386117


In [54]:
# tfidfによるスコアでの上位のキャッチコピー

score_list_10 = sorted(score_list, key=lambda x: x[2])
score_list_10.reverse()
# score_list_10.pop(4)
score_list_10 = score_list_10[:10]

print("tfidfによるスコアでの上位のキャッチコピー")
print("")
for n in score_list_10:
    print("生成例:", n[0], "\t", "tfidf:", n[2])

tfidfによるスコアでの上位のキャッチコピー

生成例: 参道も天満宮も参拝者 	 tfidf: 1.0
生成例: 参道で梅ヶ枝餅を食べながら 	 tfidf: 0.974955016027511
生成例: 参道の梅ヶ枝餅を食べながら 	 tfidf: 0.974955016027511
生成例: 参道を梅ヶ枝餅を食べながら 	 tfidf: 0.974955016027511
生成例: 参道を楽しみながらお参りし 	 tfidf: 0.9171769437986969
生成例: 参道を楽しみながらお参りを 	 tfidf: 0.9171769437986969
生成例: 参道を楽しみながらお参りを 	 tfidf: 0.9171769437986969
生成例: 参道を楽しみながらお参りし 	 tfidf: 0.9171769437986969
生成例: 参道を楽しみながらお参りに 	 tfidf: 0.9171769437986969
生成例: 参道の梅が枝餅を焼く香り 	 tfidf: 0.9003390415011042


In [55]:
# 調和平均によるスコアでの上位のキャッチコピー

score_list_10 = sorted(score_list, key=lambda x: x[3])
score_list_10.reverse()
# score_list_10.pop(4)
score_list_10 = score_list_10[:10]

print("調和平均によるスコアでの上位のキャッチコピー")
print("")
for n in score_list_10:
    print("生成例:", n[0], "\t", "調和平均:", n[3])

調和平均によるスコアでの上位のキャッチコピー

生成例: 参道も天満宮も参拝者 	 調和平均: 1.0
生成例: 参道の梅が枝餅を購入し 	 調和平均: 0.9317560696227312
生成例: 参道を梅ヶ枝餅を食べながら 	 調和平均: 0.683163792654572
生成例: 参道の梅が枝餅を焼く香り 	 調和平均: 0.588637174479906
生成例: 境内の梅が枝餅を焼く自分 	 調和平均: 0.5684361488401817
生成例: お守りも買えます長い参道に 	 調和平均: 0.5362870103453379
生成例: 熱々の梅ヶ枝餅を食べるのが 	 調和平均: 0.39290561513000016
生成例: 参道の土産物屋を見物し 	 調和平均: 0.38137167720309284
生成例: 焼きたての梅が枝餅を食べながら 	 調和平均: 0.3368964003806297
生成例: 複数の梅ヶ枝餅を食べ比べ 	 調和平均: 0.313192453060879
